In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import cv2
import albumentations as A
from tensorflow.python.client import device_lib
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
    # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)    

In [2]:
path = 'D:/ML_DL_self_study_dataset/Competition/cancer/'
train = pd.read_csv(path + 'train.csv').copy()
test = pd.read_csv(path + 'test.csv').copy()
clinical_info = pd.read_excel(path + 'clinical_info.xlsx').copy()

In [3]:
train

,ID,img_path,mask_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,...,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,N_category
0,BC_01_0001,./train_imgs/BC_01_0001.png,-,63,2015-10-23,1,2,1,19.0,2.0,...,8.0,1.0,6.0,12.0,0.0,1.0,NaN,NaN,NaN,0
1,BC_01_0002,./train_imgs/BC_01_0002.png,-,51,2015-10-28,1,1,1,22.0,3.0,...,NaN,0.0,NaN,70.0,0.0,0.0,NaN,NaN,NaN,1
2,BC_01_0003,./train_imgs/BC_01_0003.png,-,37,2015-10-29,1,2,1,NaN,2.0,...,7.0,1.0,4.0,7.0,0.0,1.0,NaN,NaN,0.0,0
3,BC_01_0004,./train_imgs/BC_01_0004.png,-,54,2016-03-08,1,2,1,0.0,3.0,...,NaN,0.0,NaN,1.0,1.0,3.0,NaN,NaN,NaN,0
4,BC_01_0005,./train_imgs/BC_01_0005.png,-,57,2015-10-30,1,2,1,8.0,2.0,...,8.0,0.0,NaN,8.0,1.0,2.0,1.0,5.44,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,BC_01_3464,./train_imgs/BC_01_3464.png,-,65,2006-12-22,1,2,1,25.0,1.0,...,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,1
996,BC_01_3482,./train_imgs/BC_01_3482.png,-,48,2006-11-17,1,1,1,7.0,1.0,...,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0
997,BC_01_3485,./train_imgs/BC_01_3485.png,-,64,2006-11-10,1,2,1,15.0,1.0,...,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,1
998,BC_01_3502,./train_imgs/BC_01_3502.png,-,50,2006-09-22,1,1,1,7.0,1.0,...,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0


In [4]:
train.isna().sum()

ID                     0
img_path               0
mask_path              0
나이                     0
수술연월일                  0
진단명                    0
암의 위치                  0
암의 개수                  0
암의 장경                 69
NG                    51
HG                    86
HG_score_1            92
HG_score_2            92
HG_score_3            89
DCIS_or_LCIS_여부        0
DCIS_or_LCIS_type    874
T_category             4
ER                     1
ER_Allred_score      297
PR                     1
PR_Allred_score      453
KI-67_LI_percent     235
HER2                  14
HER2_IHC              24
HER2_SISH            753
HER2_SISH_ratio      825
BRCA_mutation        945
N_category             0
dtype: int64

In [5]:
train.columns.unique()

Index(['ID', 'img_path', 'mask_path', '나이', '수술연월일', '진단명', '암의 위치', '암의 개수',
       '암의 장경', 'NG', 'HG', 'HG_score_1', 'HG_score_2', 'HG_score_3',
       'DCIS_or_LCIS_여부', 'DCIS_or_LCIS_type', 'T_category', 'ER',
       'ER_Allred_score', 'PR', 'PR_Allred_score', 'KI-67_LI_percent', 'HER2',
       'HER2_IHC', 'HER2_SISH', 'HER2_SISH_ratio', 'BRCA_mutation',
       'N_category'],
      dtype='object')

In [6]:
train['암의 장경'] = train['암의 장경'].fillna(train['암의 장경'].mean())
train = train.fillna(0)

test['암의 장경'] = test['암의 장경'].fillna(train['암의 장경'].mean())
test = test.fillna(0)

In [7]:
train.isna().sum()

ID                   0
img_path             0
mask_path            0
나이                   0
수술연월일                0
진단명                  0
암의 위치                0
암의 개수                0
암의 장경                0
NG                   0
HG                   0
HG_score_1           0
HG_score_2           0
HG_score_3           0
DCIS_or_LCIS_여부      0
DCIS_or_LCIS_type    0
T_category           0
ER                   0
ER_Allred_score      0
PR                   0
PR_Allred_score      0
KI-67_LI_percent     0
HER2                 0
HER2_IHC             0
HER2_SISH            0
HER2_SISH_ratio      0
BRCA_mutation        0
N_category           0
dtype: int64

In [8]:
test.isna().sum()

ID                   0
img_path             0
나이                   0
수술연월일                0
진단명                  0
암의 위치                0
암의 개수                0
암의 장경                0
NG                   0
HG                   0
HG_score_1           0
HG_score_2           0
HG_score_3           0
DCIS_or_LCIS_여부      0
DCIS_or_LCIS_type    0
T_category           0
ER                   0
ER_Allred_score      0
PR                   0
PR_Allred_score      0
KI-67_LI_percent     0
HER2                 0
HER2_IHC             0
HER2_SISH            0
HER2_SISH_ratio      0
BRCA_mutation        0
dtype: int64

In [9]:
ignore_cols_train = ['ID', 'img_path', 'mask_path', '수술연월일', 'N_category']
ignore_cols_test = ['ID', 'img_path', '수술연월일']
label = train['N_category'].copy()

train_table = train.drop(ignore_cols_train, axis = 1).copy()
test_table = test.drop(ignore_cols_test, axis = 1).copy()
train_table

,나이,진단명,암의 위치,암의 개수,암의 장경,NG,HG,HG_score_1,HG_score_2,HG_score_3,...,ER,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation
0,63,1,2,1,19.000000,2.0,1.0,2.0,2.0,1.0,...,1.0,8.0,1.0,6.0,12.0,0.0,1.0,0.0,0.00,0.0
1,51,1,1,1,22.000000,3.0,3.0,3.0,3.0,3.0,...,0.0,0.0,0.0,0.0,70.0,0.0,0.0,0.0,0.00,0.0
2,37,1,2,1,18.375564,2.0,0.0,0.0,0.0,0.0,...,1.0,7.0,1.0,4.0,7.0,0.0,1.0,0.0,0.00,0.0
3,54,1,2,1,0.000000,3.0,3.0,3.0,3.0,2.0,...,0.0,0.0,0.0,0.0,1.0,1.0,3.0,0.0,0.00,0.0
4,57,1,2,1,8.000000,2.0,2.0,3.0,2.0,1.0,...,1.0,8.0,0.0,0.0,8.0,1.0,2.0,1.0,5.44,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,65,1,2,1,25.000000,1.0,1.0,2.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
996,48,1,1,1,7.000000,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
997,64,1,2,1,15.000000,1.0,1.0,2.0,1.0,2.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
998,50,1,1,1,7.000000,1.0,1.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_table, label, random_state = 42)

In [11]:
X_train

,나이,진단명,암의 위치,암의 개수,암의 장경,NG,HG,HG_score_1,HG_score_2,HG_score_3,...,ER,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation
82,72,1,2,1,13.0,2.0,1.0,2.0,2.0,1.0,...,1.0,7.0,1.0,8.0,6.38,0.0,1.0,0.0,0.00,0.0
991,69,1,1,1,9.0,1.0,1.0,3.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.00,0.0,1.0,0.0,0.00,0.0
789,40,1,2,1,18.0,2.0,1.0,2.0,1.0,1.0,...,1.0,0.0,1.0,0.0,10.00,1.0,2.0,0.0,0.00,0.0
894,61,1,1,1,12.0,2.0,2.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0
398,40,1,1,1,1.5,1.0,1.0,3.0,1.0,1.0,...,1.0,8.0,1.0,4.0,5.00,0.0,1.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,54,1,2,1,15.0,3.0,3.0,3.0,3.0,2.0,...,1.0,7.0,1.0,7.0,24.15,0.0,2.0,0.0,1.44,0.0
270,56,1,1,1,22.0,3.0,2.0,2.0,3.0,1.0,...,1.0,8.0,1.0,4.0,11.71,0.0,2.0,0.0,1.90,0.0
860,62,1,2,1,12.0,3.0,2.0,3.0,3.0,1.0,...,1.0,0.0,1.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0
435,63,1,2,1,14.0,2.0,2.0,3.0,2.0,1.0,...,1.0,8.0,1.0,6.0,20.00,0.0,0.0,0.0,0.00,0.0


In [15]:
len(X_train.columns.unique())

23

In [16]:
len(test.columns.unique())

26

In [17]:
from xgboost import XGBRFClassifier

xgb = XGBRFClassifier(random_state = 42)
xgb.fit(X_train, y_train, eval_set = [(X_val, y_val)])

[0]	validation_0-logloss:0.48635


XGBRFClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                colsample_bylevel=1, colsample_bytree=1,
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
                importance_type=None, interaction_constraints='', max_bin=256,
                max_cat_to_onehot=4, max_delta_step=0, max_depth=6,
                max_leaves=0, min_child_weight=1, missing=nan,
                monotone_constraints='()', n_estimators=100, n_jobs=0,
                num_parallel_tree=100, objective='binary:logistic',
                predictor='auto', random_state=42, reg_alpha=0,
                sampling_method='uniform', scale_pos_weight=1, ...)

In [18]:
xgb_pred = xgb.predict(test_table)

In [19]:
xgb_pred

array([0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0])

In [23]:
output = pd.DataFrame({'ID' : test['ID'], 'N_category':xgb_pred})
output.to_csv('baseline.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [24]:
output

,ID,N_category
0,BC_01_0011,0
1,BC_01_0220,0
2,BC_01_0233,0
3,BC_01_0258,0
4,BC_01_0260,1
...,...,...
245,BC_01_3328,1
246,BC_01_3404,0
247,BC_01_3418,0
248,BC_01_3438,0
